In [1]:
"""credit.policy: 
    -->> 1 if the customer meets the credit underwriting criteria of LendingClub.com, and 0 otherwise.

purpose: The purpose of the loan (takes values "credit_card", "debt_consolidation", "educational", "major_purchase", "small_business", and "all_other").

int.rate: The interest rate of the loan, as a proportion (a rate of 11% would be stored as 0.11). Borrowers judged by LendingClub.com to be more risky are assigned higher interest rates.
    -->> higher the interest rate that means risk is higher
    
installment: The monthly installments owed by the borrower if the loan is funded.

log.annual.inc: The natural log of the self-reported annual income of the borrower.

dti: The debt-to-income ratio of the borrower (amount of debt divided by annual income).

fico: The FICO credit score of the borrower.

days.with.cr.line: The number of days the borrower has had a credit line.

revol.bal: The borrower's revolving balance (amount unpaid at the end of the credit card billing cycle).

revol.util: The borrower's revolving line utilization rate (the amount of the credit line used relative to total credit available).

inq.last.6mths: The borrower's number of inquiries by creditors in the last 6 months.

delinq.2yrs: The number of times the borrower had been 30+ days past due on a payment in the past 2 years.

pub.rec: The borrower's number of derogatory public records (bankruptcy filings, tax liens, or judgments).

not.fully.paid: whether or not the borrower paid back their loan in full
"""

 

'credit.policy: 1 if the customer meets the credit underwriting criteria of LendingClub.com, and 0 otherwise.\n\npurpose: The purpose of the loan (takes values "credit_card", "debt_consolidation", "educational", "major_purchase", "small_business", and "all_other").\n\nint.rate: The interest rate of the loan, as a proportion (a rate of 11% would be stored as 0.11). Borrowers judged by LendingClub.com to be more risky are assigned higher interest rates.\n\ninstallment: The monthly installments owed by the borrower if the loan is funded.\n\nlog.annual.inc: The natural log of the self-reported annual income of the borrower.\n\ndti: The debt-to-income ratio of the borrower (amount of debt divided by annual income).\n\nfico: The FICO credit score of the borrower.\n\ndays.with.cr.line: The number of days the borrower has had a credit line.\n\nrevol.bal: The borrower\'s revolving balance (amount unpaid at the end of the credit card billing cycle).\n\nrevol.util: The borrower\'s revolving line 

In [41]:
## Necessary functions
def get_new_column_names(column_names):
    try:
        columns_dict = {"credit.policy":"credit_policy",
        "purpose":"purpose",
        "int.rate":"interest_rate",
        "installment":"monthly_installment_of_users",
        "log.annual.inc":"annual_income_in",
        "dti":"debt_to_income_ration",
        "fico":"fico_credit_score",
        "days.with.cr.line":"days_with_cr_line",
        "revol.bal":"amount_unpaid_past_billing_cycle",
        "revol.util":"revol_util",
        "inq.last.6mths":"creditors_inquery_last_6_months",
        "delinq.2yrs":"count_of_late_payment_past_2_years",
        "pub.rec": "negative_public_records",
        "not.fully.paid" :"paid_fully_or_not"}
        print(type(column_names))
        for column_name_index in range(len(column_names)):
            column_names[column_name_index] = columns_dict[column_names[column_name_index].strip().lower()]
        return column_names
    except Exception as e:
        raise Exception("Error has encountered at function ==>> get_new_column_names", e)



<class 'list'>


Exception: ('Error has encountered at function ==>> get_new_column_names', KeyError('credit_policy'))

In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

In [39]:
#import the dataset
dataframe = pd.read_csv("C:/Users/Admin/Desktop/bits-AIML/machine_learning_practice_codes_github_folder/machine_learning/Projects/Decision_tree_model_for_loan_dataset/loan_data.csv")
dataframe.columns = get_new_column_names(list(dataframe.columns))

<class 'list'>


In [36]:
get_new_column_names(list(dataframe.columns))

<class 'list'>


['credit_policy',
 'purpose',
 'interest_rate',
 'monthly_installment_of_users',
 'annual_income_in',
 'debt_to_income_ration',
 'fico_credit_score',
 'days_with_cr_line',
 'amount_unpaid_past_billing_cycle',
 'revol_util',
 'creditors_inquery_last_6_months',
 'count_of_late_payment_past_2_years',
 'negative_public_records',
 'paid_fully_or_not']

In [40]:
dataframe


,credit_policy,purpose,interest_rate,monthly_installment_of_users,annual_income_in,debt_to_income_ration,fico_credit_score,days_with_cr_line,amount_unpaid_past_billing_cycle,revol_util,creditors_inquery_last_6_months,count_of_late_payment_past_2_years,negative_public_records,paid_fully_or_not
0,1,debt_consolidation,0.1189,829.10,11.350407,19.48,737,5639.958333,28854,52.1,0,0,0,0
1,1,credit_card,0.1071,228.22,11.082143,14.29,707,2760.000000,33623,76.7,0,0,0,0
2,1,debt_consolidation,0.1357,366.86,10.373491,11.63,682,4710.000000,3511,25.6,1,0,0,0
3,1,debt_consolidation,0.1008,162.34,11.350407,8.10,712,2699.958333,33667,73.2,1,0,0,0
4,1,credit_card,0.1426,102.92,11.299732,14.97,667,4066.000000,4740,39.5,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9573,0,all_other,0.1461,344.76,12.180755,10.39,672,10474.000000,215372,82.1,2,0,0,1
9574,0,all_other,0.1253,257.70,11.141862,0.21,722,4380.000000,184,1.1,5,0,0,1
9575,0,debt_consolidation,0.1071,97.81,10.596635,13.09,687,3450.041667,10036,82.9,8,0,0,1
9576,0,home_improvement,0.1600,351.58,10.819778,19.18,692,1800.000000,0,3.2,5,0,0,1
